In [9]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, IsNullCondition
import cv2
import torch
import clip
from PIL import Image
import numpy as np
from pathlib import Path
from typing import List, Tuple, Union, Dict
# from boxmot.appearance.reid_auto_backend import ReidAutoBackend
from boxmot.appearance.reid.auto_backend import ReidAutoBackend
import os
from transformers import pipeline
from helpers import extract_frame_by_count, get_person_crop_from_coords


## Config

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))  # Should show your GPU model
print(torch.version.cuda)  # Should display the installed CUDA version


True
0
1
NVIDIA GeForce RTX 3050 Laptop GPU
12.1


In [10]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Path to your local Qdrant database
QDRANT_DB_PATH = "./qdrant_local_db"
COLLECTION_NAME = "stream"

# Maps cam_id to video file paths
VIDEO_LOCATION_MAPPING = {
    0: r"C:\Users\hthek\OneDrive\Desktop\Collected Dataset\Vid_3_cam_0.mp4", 
    1: r"C:\Users\hthek\OneDrive\Desktop\Collected Dataset\Vid_3_cam_1.mp4", 
    2: r'C:\Users\hthek\OneDrive\Desktop\Collected Dataset\Vid_3_cam_1.mp4'
}

CROP_SAVE_DIRECTORY = r"C:\Users\hthek\Downloads\Capstone-AI-SE\Demo\saved_crops_qdrant"
osnet_weights = Path(r"C:\Users\hthek\Downloads\Capstone-AI-SE\MCDPT\osnet_x0_25_msmt17.pt")

# Dictionary mapping CLIP model names to embedding dimensions
CLIP_EMBEDDING_SIZES = {
    'RN50': 1024,
    'RN101': 512,
    'RN50x4': 640,
    'RN50x16': 768,
    'RN50x64': 1024,
    'ViT-B/32': 512,
    'ViT-B/16': 512,
    'ViT-L/14': 768,
    'ViT-L/14@336px': 768
}

# Extract embedding size based on model name
CLIP_MODEL_NAME = 'ViT-B/32'  # or whatever model you're using
EMBEDDING_SIZE = CLIP_EMBEDDING_SIZES[CLIP_MODEL_NAME]

ELDERLY_THRESHOLD = 0.6
DISABLED_THRESHOLD = 0.6
EMOTION_THRESHOLD = 0.6  # Min confidence for "sad", "fear", or "angry" -> potentiallyLost = True

## Reading & Accessing Existing QDrant DB

In [11]:
# 1. Connect to existing local storage
client = QdrantClient("http://localhost:6333")

In [ ]:
try:
    client.create_collection(
        collection_name='CLIP_embeddings',
        vectors_config=VectorParams(size=EMBEDDING_SIZE, distance=Distance.COSINE)
    )
except:
    print()



1


In [8]:
# 2. Verifying local qdrant local database
records = client.retrieve(collection_name="CLIP_embeddings", ids=range(100), with_payload=True, with_vectors=True)
records

[Record(id=0, payload={'Pid': '1-1', 'is_lost': True, 'is_elderly': False, 'is_disabled': False}, vector=[0.009810134, -0.011335815, -0.03451092, -0.014806742, 0.009985587, 0.006964737, -0.005263602, 0.1083234, 0.051812153, 0.03139853, 0.026668917, 0.043359876, 0.024243083, -0.0112900445, -0.028606532, -0.014280382, 0.004088827, 0.0018317717, 0.015714522, 0.004588488, -0.010878111, 0.006491776, -0.016599417, 0.0093982, 0.008948123, 0.007761906, -0.00013349716, -0.021359544, 0.003789412, -0.0007642712, 0.013975245, -0.028652303, 0.016935067, -0.0124648195, -0.020688245, 0.0067663984, -0.008467534, -0.012769956, -0.006735885, -0.014265125, 0.0045770453, -0.030803515, -0.017285975, -0.006011186, -0.008398878, 0.16965581, 0.0003666404, 0.0111756185, -0.0075864526, -0.03533479, -0.00656806, 0.0076856217, 0.0036826143, -0.02032208, 0.0057823337, 0.0019662224, -0.019971173, 0.008787927, 0.0010994445, -0.019589754, 0.069876224, -0.030132215, -0.004950837, -0.021817248, 0.007895403, -0.01356331

In [25]:
# Show first few records
print("\nFirst 3 records:")
for r in records[:100]:
    print(f"  ID: {r.id}, Payload: {r.payload}")

# Show summary by Pid and cam_id
from collections import Counter
pids = [r.payload['Pid'] for r in records]
cam_ids = [r.payload['cam_id'] for r in records]
print(f"\nSummary:")
print(f"  PIDs: {dict(Counter(pids))}")
print(f"  Camera IDs: {dict(Counter(cam_ids))}")


First 3 records:
  ID: 415, Payload: {'Pid': '12-1'}
  ID: 416, Payload: {'Pid': '12-1'}
  ID: 417, Payload: {'Pid': '4-0'}
  ID: 418, Payload: {'Pid': '4-0'}
  ID: 419, Payload: {'Pid': '4-0'}
  ID: 420, Payload: {'Pid': '4-0'}
  ID: 421, Payload: {'Pid': '4-0'}
  ID: 422, Payload: {'Pid': '4-0'}
  ID: 423, Payload: {'Pid': '4-0'}
  ID: 424, Payload: {'Pid': '4-0'}
  ID: 425, Payload: {'Pid': '4-0'}
  ID: 426, Payload: {'Pid': '4-0'}
  ID: 427, Payload: {'Pid': '4-0'}
  ID: 428, Payload: {'Pid': '4-0'}
  ID: 429, Payload: {'Pid': '4-0'}
  ID: 430, Payload: {'Pid': '4-0'}
  ID: 431, Payload: {'Pid': '4-0'}
  ID: 432, Payload: {'Pid': '4-0'}
  ID: 433, Payload: {'Pid': '4-0'}
  ID: 434, Payload: {'Pid': '4-0'}
  ID: 435, Payload: {'Pid': '4-0'}
  ID: 436, Payload: {'Pid': '4-0'}
  ID: 437, Payload: {'Pid': '4-0'}
  ID: 438, Payload: {'Pid': '4-0'}
  ID: 439, Payload: {'Pid': '4-0'}
  ID: 440, Payload: {'Pid': '1-1'}
  ID: 441, Payload: {'Pid': '1-1'}
  ID: 442, Payload: {'Pid': '1-1'}


KeyError: 'cam_id'

## CLIP

In [7]:
class CLIP:
    """
    CLIP-based Person Re-Identification system.
    Uses text descriptions to retrieve matching persons from an image gallery.
    """
    
    def __init__(self, model_name: str = "ViT-B/32", device: str = None):
        """
        Initialize CLIP model for ReID.
        
        Args:
            model_name: CLIP model variant. Options: "RN50", "RN101", "RN50x4", 
                       "RN50x16", "RN50x64", "ViT-B/32", "ViT-B/16", "ViT-L/14"
            device: Device to run model on. If None, auto-detects GPU/CPU.
        """
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Loading CLIP model: {model_name} on {self.device}")
        
        # Load CLIP model and preprocessing
        self.model, self.preprocess = clip.load(model_name, device=self.device)
        self.model.eval()
        
        print(f"Model loaded successfully!")
    
    def encode_images(self, image_paths: List[Union[str, Path]]) -> torch.Tensor:
        """
        Encode a list of images into feature vectors.
        
        Args:
            image_paths: List of paths to image files
            
        Returns:
            Normalized image feature tensor of shape (N, feature_dim)
        """
        image_features = []
        
        for img_path in image_paths:
            try:
                image = Image.open(img_path).convert("RGB")
                image_input = self.preprocess(image).unsqueeze(0).to(self.device)
                
                with torch.no_grad():
                    features = self.model.encode_image(image_input)
                    features = features / features.norm(dim=-1, keepdim=True)  # Normalize
                    
                image_features.append(features)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                continue
        
        if not image_features:
            raise ValueError("No images were successfully processed")
            
        return torch.cat(image_features, dim=0)
    
    def encode_person_crop(self, image: Union[Image.Image, np.ndarray, torch.Tensor]) -> torch.Tensor:
        """
        Encode a single person image into a feature vector.
        
        Args:
            image: Input image (PIL cropped person image)
            
        Returns:
            Normalized image feature tensor of shape (1, feature_dim)
        """
        try:
            # Preprocess and encode
            image_input = self.preprocess(image).unsqueeze(0).to(self.device)
            
            with torch.no_grad():
                features = self.model.encode_image(image_input)
                features = features / features.norm(dim=-1, keepdim=True)  # Normalize
                features = features.squeeze(0)
            
            return features
            
        except Exception as e:
            raise ValueError(f"Failed to encode single image: {e}")
    
    def encode_text(self, text_descriptions: List[str]) -> torch.Tensor:
        """
        Encode text descriptions into feature vectors.
        
        Args:
            text_descriptions: List of text descriptions
            
        Returns:
            Normalized text feature tensor of shape (N, feature_dim)
        """
        text_tokens = clip.tokenize(text_descriptions).to(self.device)
        
        with torch.no_grad():
            text_features = self.model.encode_text(text_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
        return text_features
    
    def retrieve(self, 
                 text_query: str,
                 image_paths: List[Union[str, Path]],
                 top_k: int = 10,
                 threshold: float = None) -> List[Tuple[str, float]]:
        """
        Retrieve images matching the text description.
        
        Args:
            text_query: Text description of the person to find
            image_paths: List of paths to gallery images
            top_k: Number of top matches to return
            threshold: Optional similarity threshold (0-1). Only return matches above this.
            
        Returns:
            List of tuples (image_path, similarity_score) sorted by similarity
        """
        print(f"Encoding {len(image_paths)} images...")
        image_features = self.encode_images(image_paths)
        
        print(f"Encoding text query: '{text_query}'")
        text_features = self.encode_text([text_query])
        
        # Compute cosine similarity
        similarities = (image_features @ text_features.T).squeeze().cpu().numpy()
        
        # Handle single image case
        if similarities.ndim == 0:
            similarities = np.array([similarities])
        
        # Get top-k indices
        top_k = min(top_k, len(image_paths))
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        # Prepare results
        results = []
        for idx in top_indices:
            score = float(similarities[idx])
            if threshold is None or score >= threshold:
                results.append((str(image_paths[idx]), score))
        
        return results
    
    def batch_retrieve(self,
                      text_queries: List[str],
                      image_paths: List[Union[str, Path]],
                      top_k: int = 10,
                      threshold: float = None) -> dict:
        """
        Retrieve images for multiple text queries at once.
        
        Args:
            text_queries: List of text descriptions
            image_paths: List of paths to gallery images
            top_k: Number of top matches to return per query
            threshold: Optional similarity threshold (0-1). Only return matches above this.
            
        Returns:
            Dictionary mapping each query to its list of (image_path, score) tuples
        """
        print(f"Encoding {len(image_paths)} images...")
        image_features = self.encode_images(image_paths)
        
        print(f"Encoding {len(text_queries)} text queries...")
        text_features = self.encode_text(text_queries)
        
        # Compute all similarities at once
        similarities = (image_features @ text_features.T).cpu().numpy()  # Shape: (n_images, n_queries)
        
        results = {}
        for i, query in enumerate(text_queries):
            query_sims = similarities[:, i]
            top_k_actual = min(top_k, len(image_paths))
            top_indices = np.argsort(query_sims)[::-1][:top_k_actual]
            
            results[query] = [
                (str(image_paths[idx]), float(query_sims[idx]))
                for idx in top_indices
                if (threshold is None or float(query_sims[idx]) >= threshold)
            ]
        
        return results

In [8]:
REID_SYSTEM = CLIP(model_name=CLIP_MODEL_NAME)

Loading CLIP model: ViT-B/32 on cuda
Model loaded successfully!


## Load emotion classification model

In [9]:
def load_emotion_model(device_num=0):
    """
    Loads the emotion classification model once.
    """
    print("Loading emotion detection model...")
    emotion_clf = pipeline(
        "image-classification", 
        model="trpakov/vit-face-expression", 
        device=device_num if device_num >= 0 and torch.cuda.is_available() else -1
    )
    print("Emotion model loaded successfully.")
    return emotion_clf

### Functions for determing emotion & lost status

In [10]:
def run_emotion_model(crop_image, emotion_clf):
    """
    Runs emotion detection on a cropped PIL Image.
    Returns the top prediction as a dict with 'label' and 'score'.
    """
    try:
        pred = emotion_clf(crop_image, top_k=1)
        if pred:
            return pred[0]  # {'label': '...', 'score': ...}
    except Exception as e:
        print(f"  Error in emotion model: {e}")
    
    return {"label": "unknown", "score": 0.0}

In [11]:

def is_potentially_lost(emotion_result, threshold=0.6):
    """
    Determines if a person is potentially lost based on emotion.
    Returns True if emotion is "sad" or "fear" with confidence above threshold.
    """
    label = emotion_result['label'].lower()
    score = emotion_result['score']
    
    if label in ["sad", "fear","angry"] and score > threshold:
        return True
    return False

## Creating a Qdrant collection (table) for the text embeddings

In [ ]:
client.create_collection(
    collection_name='text_embeddings',
    vectors_config=VectorParams(size=EMBEDDING_SIZE, distance=Distance.COSINE)
)

texts = [
    "an elderly person over 60 years old",
    "a person in a wheel chair"
]

prompt_embeddings = REID_SYSTEM.encode_text(texts).to('cpu')

elderly_person_embedding = prompt_embeddings[0].numpy()
disabled_person_embedding = prompt_embeddings[1].numpy()

print(prompt_embeddings.shape)

points = [
    PointStruct(id=1, vector=elderly_person_embedding, payload={'title': 'elderly person prompt embedding'}),
    PointStruct(id=2, vector=disabled_person_embedding, payload={'title': 'disabled person prompt embedding'})
]

client.upsert(collection_name='text_embeddings', points=points)

torch.Size([2, 512])


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Functions to determine if a person is elderly or disabled

In [13]:
def determine_elderly(person_emb, threshold=0.6):
    # Retrieve specific record by ID
    elderly_result = client.retrieve(
        collection_name="text_embeddings",
        ids=[1], # id 1 is the elderly person embedding
        with_vectors=True
    )

    # Extract the vector and payload
    if elderly_result:
        elderly_point = elderly_result[0]
        elderly_embedding = elderly_point.vector
        elderly_payload = elderly_point.payload
        print(f"ID: {elderly_point.id}")
        print(f"Vector: {elderly_embedding}")
        print(f"Payload: {elderly_payload}")
    else:
        print("No elderly person embedding found.")
        return False
    
    cos_sim = np.dot(person_emb, elderly_embedding) # Because it's already normalized, cosine similarity is the dot product
    print(f"Cosine Similarity: {cos_sim}")
    
    if cos_sim > threshold:
        return True
    return False
        

In [14]:
def determine_disabled(person_emb, threshold=0.6):
    # Retrieve specific record by ID
    disabled_result = client.retrieve(
        collection_name="text_embeddings",
        ids=[2], # id 2 is the disabled person embedding
        with_vectors=True
    )

    # Extract the vector and payload
    if disabled_result:
        disabled_point = disabled_result[0]
        disabled_embedding = disabled_point.vector
        disabled_payload = disabled_point.payload
        print(f"ID: {disabled_point.id}")
        print(f"Vector: {disabled_embedding}")
        print(f"Payload: {disabled_payload}")
    else:
        print("No disabled person embedding found.")
        return False
    
    cos_sim = np.dot(person_emb, disabled_embedding) # Because it's already normalized, cosine similarity is the dot product
    print(f"Cosine Similarity: {cos_sim}")
    
    if cos_sim > threshold:
        return True
    return False


In [15]:
prompt_records = client.scroll(collection_name='text_embeddings', limit=100, with_payload=True, with_vectors=True)[0]
print(f"\nTotal records in database: {len(prompt_records)}")

# Show first few records
print(f"\nRecords:")
for r in prompt_records[:3]:
    print(f"  ID: {r.id}, Payload: {r.payload}")


Total records in database: 2

Records:
  ID: 1, Payload: {'title': 'elderly person prompt embedding'}
  ID: 2, Payload: {'title': 'disabled person prompt embedding'}


## Creating a Qdrant collection for CLIP & OSnet embeddings

In [16]:
client.delete_collection(collection_name='CLIP_embeddings')
client.delete_collection(collection_name='osnet_embeddings')

client.create_collection(
    collection_name='CLIP_embeddings',
    vectors_config=VectorParams(size=EMBEDDING_SIZE, distance=Distance.COSINE)
)

client.create_collection(
    collection_name='osnet_embeddings',
    vectors_config=VectorParams(size=EMBEDDING_SIZE, distance=Distance.COSINE)
)

True

### Create OSNet embedding

In [17]:
def get_osnet_embedding(pil_image):
    # Convert PIL to numpy array (RGB format, which PIL uses by default)
    image_np = np.array(pil_image)

    # Initialize model
    model = ReidAutoBackend(
        weights=osnet_weights,
        device=DEVICE,
        half=False
    )

    # Get the actual model and move to device
    osnet_model = model.model.model.to(DEVICE)
    
    # Convert image to tensor and preprocess
    from torchvision import transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Preprocess image
    image_tensor = transform(image_np).unsqueeze(0).to(DEVICE)
    
    # Get embedding
    with torch.no_grad():
        osnet_embedding = osnet_model(image_tensor).cpu().numpy()[0]

    # Normalize
    osnet_embedding = osnet_embedding / np.linalg.norm(osnet_embedding)

    print(f"OSNet Embedding shape: {osnet_embedding.shape}")
    return osnet_embedding

## MAIN

In [18]:
def main():
    # Create crop save directory
    os.makedirs(CROP_SAVE_DIRECTORY, exist_ok=True)
    
    # 2. Load emotion model
    try:
        emotion_clf = load_emotion_model(device_num=0)
    except Exception as e:
        print(f"FATAL: Could not load emotion model: {e}")
        return
    
    # 3. Get records where potentiallyLost is None
    print(f"\nQuerying records where 'potentiallyLost' is None...")
    
    all_records = client.scroll(
        collection_name=COLLECTION_NAME,
        with_payload=True,
        with_vectors=False,
        limit=1000  # Adjust as needed
    )[0]
    
    # Filter for records where potentiallyLost is None
    records_to_process = [
        record for record in all_records 
        if record.payload.get('potentiallyLost') is None
    ]
    
    print(f"Found {len(records_to_process)} records to process.")
    
    if not records_to_process:
        print("No records to process. Exiting.")
        return
    
    # 4. Process each record
    count = 0
    for record in records_to_process:
        print(f"\n--- Processing Record ID: {record.id} ---")
        
        # Extract payload data
        payload = record.payload
        cam_id = payload['cam_id']
        coords = payload['coords']  # [x1, y1, x2, y2]
        frame_count = payload['frame_count']
        pid = payload['Pid']
        
        # A. Get video path
        video_path = VIDEO_LOCATION_MAPPING.get(cam_id)
        if not video_path:
            print(f"  Error: No video path found for cam_id {cam_id}. Skipping.")
            continue
                
        # B. Extract frame
        frame, frame_w, frame_h = extract_frame_by_count(video_path, frame_count)
        if frame is None:
            print("  Skipping record.")
            continue
        
        # C. Crop the person
        crop_image = get_person_crop_from_coords(frame, coords, frame_w, frame_h)
        if crop_image is None:
            print("  Skipping record.")
            continue
        
        # D. Save crop (optional, for debugging)
        try:
            save_path = os.path.join(CROP_SAVE_DIRECTORY, f"crop_id_{record.id}.jpg")
            crop_image.save(save_path)
            print(f"  Saved crop to: {save_path}")
        except Exception as e:
            print(f"  Warning: Could not save crop: {e}")
            
            
        # E. Run emotion model
        print("  Running emotion analysis...")
        emotion_result = run_emotion_model(crop_image, emotion_clf)
        print(f"  Emotion Result: {emotion_result}")
        
        # F. Determine potentiallyLost status
        is_lost = is_potentially_lost(emotion_result, EMOTION_THRESHOLD)
        
        # G. Create an embedding for the person
        clip_person_emb = REID_SYSTEM.encode_person_crop(crop_image).to('cpu').numpy()
        print(clip_person_emb.shape)
        
        osnet_person_emb = get_osnet_embedding(crop_image)
        
        # H. Determine Elderly status
        is_elderly = determine_elderly(clip_person_emb, ELDERLY_THRESHOLD)
        
        # I. Determine Disabled status
        is_disabled = determine_disabled(clip_person_emb, DISABLED_THRESHOLD)
        
        
        # J. Update Qdrant record
        try:
            client.set_payload(
                collection_name=COLLECTION_NAME,
                payload={"isElderly": is_elderly, "isDisabled": is_disabled, "potentiallyLost": is_lost},
                points=[record.id]
            )
            print(f"  ✓ Updated isElderly = {is_elderly} and isDisabled = {is_disabled} and potentiallyLost = {is_lost} for Record ID: {record.id}")
        except Exception as e:
            print(f"  Error updating record: {e}")
            
        # K. Add record to CLIP & OSNet collections
        clip_points = [PointStruct(id=record.id, vector=clip_person_emb, payload={"Pid": pid})]
        osnet_points = [PointStruct(id=record.id, vector=osnet_person_emb, payload={"Pid": pid})]
        
        client.upsert(collection_name='CLIP_embeddings', points=clip_points)
        client.upsert(collection_name='osnet_embeddings', points=osnet_points)

        count+=1
        if count >= 100:
            break
        
    print("\n--- Processing Complete ---")

In [19]:
if __name__ == "__main__":
    main()

Loading emotion detection model...


c:\Users\hthek\anaconda3\envs\tracking\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Emotion model loaded successfully.

Querying records where 'potentiallyLost' is None...
Found 585 records to process.

--- Processing Record ID: 415 ---
  Successfully extracted frame at count 20.
  Saved crop to: C:\Users\hthek\Downloads\Capstone-AI-SE\Demo\saved_crops_qdrant\crop_id_415.jpg
  Running emotion analysis...
  Emotion Result: {'score': 0.5734620690345764, 'label': 'sad'}
(512,)
OSNet Embedding shape: (512,)
ID: 1
Vector: [0.011039811, -0.017242553, -0.004940068, -0.0054932027, -0.012786956, -0.032440413, 0.013580418, -0.11712729, 0.017913945, 0.00739675, 0.020248555, 0.015960805, -0.016906857, -0.024200609, 0.011581502, -0.0016498682, 0.017395142, 0.014175515, -0.028442582, -0.028976643, 0.012451259, 0.012207117, -0.0038528712, -0.04824863, -0.018997326, -0.009346074, 0.016265983, 0.034393553, 0.009941171, -0.017089965, 0.00094605156, 0.018890513, 0.0026512332, 0.036285657, -0.003835705, -0.0259859, 0.0050545093, -0.00095606525, 0.008407651, -0.0033931972, -0.012573331, -

c:\Users\hthek\anaconda3\envs\tracking\lib\site-packages\transformers\pipelines\base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


(512,)
OSNet Embedding shape: (512,)
ID: 1
Vector: [0.011039811, -0.017242553, -0.004940068, -0.0054932027, -0.012786956, -0.032440413, 0.013580418, -0.11712729, 0.017913945, 0.00739675, 0.020248555, 0.015960805, -0.016906857, -0.024200609, 0.011581502, -0.0016498682, 0.017395142, 0.014175515, -0.028442582, -0.028976643, 0.012451259, 0.012207117, -0.0038528712, -0.04824863, -0.018997326, -0.009346074, 0.016265983, 0.034393553, 0.009941171, -0.017089965, 0.00094605156, 0.018890513, 0.0026512332, 0.036285657, -0.003835705, -0.0259859, 0.0050545093, -0.00095606525, 0.008407651, -0.0033931972, -0.012573331, -0.018356452, 0.01754773, -0.017089965, -0.0157777, 0.03137229, -0.025665464, -0.006748247, -0.017364625, -0.0021267086, 0.0014267068, -0.028091628, -0.013946631, 0.016525384, -0.02072158, -0.0056801243, 0.01950087, -0.018707408, -0.006725359, 0.010490491, 0.05267371, -0.021804962, -0.02308671, -0.006633805, -0.017578248, -0.00767141, -0.032318342, 0.03610255, 0.015640369, -0.021194607,

In [ ]:
# 2. Verifying local qdrant local database
records = client.scroll(collection_name=COLLECTION_NAME, limit=100, with_payload=True, with_vectors=True)[0]
print(f"\nTotal records in database: {len(records)}")

# Show first few records
print("\nRecords:")
for r in records[:10]:
    print(f"  ID: {r.id}, Payload: {r.payload}")

# Show summary by Pid and cam_id
from collections import Counter
pids = [r.payload['Pid'] for r in records]
cam_ids = [r.payload['cam_id'] for r in records]
print(f"\nSummary:")
print(f"  PIDs: {dict(Counter(pids))}")
print(f"  Camera IDs: {dict(Counter(cam_ids))}")


Total records in database: 100

First records:
  ID: 0, Payload: {'Pid': '1-0', 'coords': [634, 483, 681, 600], 'cam_id': 0, 'frame_count': 20, 'cross_cam': False, 'isElderly': False, 'isDisabled': False, 'potentiallyLost': False}
  ID: 1, Payload: {'Pid': '1-0', 'coords': [634, 483, 681, 600], 'cam_id': 0, 'frame_count': 20, 'cross_cam': False, 'isElderly': False, 'isDisabled': False, 'potentiallyLost': False}
  ID: 2, Payload: {'Pid': '1-0', 'coords': [634, 483, 681, 600], 'cam_id': 0, 'frame_count': 20, 'cross_cam': False, 'isElderly': False, 'isDisabled': False, 'potentiallyLost': False}
  ID: 3, Payload: {'Pid': '1-0', 'coords': [634, 483, 681, 600], 'cam_id': 0, 'frame_count': 20, 'cross_cam': False, 'isElderly': False, 'isDisabled': False, 'potentiallyLost': False}
  ID: 4, Payload: {'Pid': '1-0', 'coords': [634, 483, 681, 600], 'cam_id': 0, 'frame_count': 20, 'cross_cam': False, 'isElderly': False, 'isDisabled': False, 'potentiallyLost': False}
  ID: 5, Payload: {'Pid': '1-0',

In [24]:
# 2. Verifying local qdrant local database
records = client.scroll(collection_name='CLIP_embeddings', limit=100, with_payload=True, with_vectors=True)[0]
print(f"\nTotal records in database: {len(records)}")

# Show first few records
print("\nRecords:")
for r in records[:10]:
    print(f"  ID: {r.id}, Payload: {r.payload}, Vector:{r.vector}")


Total records in database: 100

Records:
  ID: 415, Payload: {'Pid': '12-1'}, Vector:[0.020904642, -0.007156406, -0.034423992, -0.014991832, 0.012764038, 0.010681204, 0.010505727, 0.04461691, 0.053955335, 0.013954229, 0.0037651241, -0.003400819, 0.03125015, -0.024124261, -0.04025288, 0.0052490486, -0.003925342, 0.02078257, 0.012908997, 0.023285024, -0.042389117, 0.026123172, 0.0037212549, -0.0010356952, -0.001479156, 0.009223982, 0.011100822, -0.00636676, 0.0018291561, -0.014465401, 0.017227255, -0.041595656, -0.009269758, -0.007858314, -0.025390746, -0.0361635, -0.0076218015, -0.04037495, -0.028198376, 0.0514834, -0.021301372, -0.027298104, 0.005870847, -0.01098638, -0.016143875, 0.12146054, -0.0053711194, 0.012619079, 0.023773307, -0.0351259, 0.02667249, 0.018402187, -0.009353682, 0.018982025, -0.0059738443, -0.016967854, 0.03555315, 0.014846873, -0.00034117862, -0.019226165, 0.10583547, -0.0008750004, -0.013114992, -0.031280667, -0.0007205044, -0.013839788, -0.027267586, -0.0197754